In [ ]:
# =============================================================================
# TEXTURE PREPROCESSING TOOL - AI5 DATASET
# Navigate through t1.png to t17.png files in datasets/tex2 folder
# Select best areas and orientations for stick generation
# =============================================================================

import cv2
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
from pathlib import Path

matplotlib.use('TkAgg')  # Ensure GUI popup works

# --- Config ---
texture_base_path = r"datasets/tex2"
output_config_file = r"datasets/AI5/texture_selections_config.json"

# Stick dimensions (matching your generator)
stick_w, stick_h = 222, 1766    # Actual stick size for visual guide
crop_w, crop_h = 300, 400        # Crop size from texture (can be larger than stick)

# Global state
current_texture_num = 1
texture_data = {}  # Store all texture configurations
current_img = None
current_rotation = 0  # 0, 90, 180, 270 degrees
cursor_scale = 1.0   # Scale factor for cursor size
clicks = []          # Store clicks for current texture

# Initialize plot
fig, ax = plt.subplots(figsize=(14, 10))
plt.subplots_adjust(bottom=0.1)

# Create cursor box (hollow stick shape)
cursor_box = ax.add_patch(
    plt.Rectangle((0, 0), stick_w * cursor_scale, stick_h * cursor_scale, 
                 linewidth=2, edgecolor='red', facecolor='none', visible=False)
)

def load_texture(texture_num):
    """Load t{num}.png file"""
    global current_img, current_rotation
    
    texture_path = os.path.join(texture_base_path, f"t{texture_num}.png")
    
    if not os.path.exists(texture_path):
        print(f"Texture {texture_num} not found: {texture_path}")
        return False
    
    try:
        img = cv2.imread(texture_path)
        if img is None:
            print(f"Could not load texture {texture_num}")
            return False
            
        current_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        current_rotation = 0  # Reset rotation
        return True
    except Exception as e:
        print(f"Error loading texture {texture_num}: {e}")
        return False

def rotate_image(img, angle):
    """Rotate image by specified angle (0, 90, 180, 270)"""
    if angle == 0:
        return img
    elif angle == 90:
        return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(img, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        return img

def update_display():
    """Update the display with current texture and rotation"""
    global current_img, current_rotation
    
    if current_img is None:
        return
    
    # Apply rotation
    display_img = rotate_image(current_img, current_rotation)
    
    # Clear and redraw
    ax.clear()
    ax.imshow(display_img)
    ax.set_title(f"Texture t{current_texture_num}.png | Rotation: {current_rotation}° | Scale: {cursor_scale:.1f}x\n"
                f"Click: Select area | →: Rotate 90° | ↑↓: Scale cursor | Space: Next texture | Enter: Save & Exit")
    
    # Recreate cursor box with current scale
    global cursor_box
    cursor_box = ax.add_patch(
        plt.Rectangle((0, 0), stick_w * cursor_scale, stick_h * cursor_scale, 
                     linewidth=2, edgecolor='red', facecolor='none', visible=False)
    )
    
    # Redraw existing clicks for this texture
    if str(current_texture_num) in texture_data:
        for click_data in texture_data[str(current_texture_num)]:
            cx, cy = click_data['x'], click_data['y']
            ax.plot(cx, cy, 'go', markersize=8)
    
    fig.canvas.draw()

def save_texture_config():
    """Save all texture configurations to JSON file"""
    try:
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(output_config_file)
        os.makedirs(output_dir, exist_ok=True)
        
        with open(output_config_file, 'w') as f:
            json.dump(texture_data, f, indent=2)
        print(f"Saved texture config to: {output_config_file}")
        return True
    except Exception as e:
        print(f"Error saving config: {e}")
        return False

def load_texture_config():
    """Load existing texture configurations"""
    global texture_data
    try:
        if os.path.exists(output_config_file):
            with open(output_config_file, 'r') as f:
                texture_data = json.load(f)
            print(f"Loaded existing config from: {output_config_file}")
        else:
            texture_data = {}
            print("Starting with empty config")
    except Exception as e:
        print(f"Error loading config: {e}")
        texture_data = {}

def next_texture():
    """Move to next texture"""
    global current_texture_num
    
    # Find next available texture (t1.png to t17.png)
    for i in range(current_texture_num + 1, 18):  # Try 1-17
        if load_texture(i):
            current_texture_num = i
            update_display()
            print(f"Loaded texture t{current_texture_num}.png")
            return
    
    # If no more textures, wrap to beginning
    for i in range(1, current_texture_num):
        if load_texture(i):
            current_texture_num = i
            update_display()
            print(f"Wrapped to texture t{current_texture_num}.png")
            return
    
    print("No more textures found")

def previous_texture():
    """Move to previous texture"""
    global current_texture_num
    
    # Find previous available texture
    for i in range(current_texture_num - 1, 0, -1):  # Try backwards
        if load_texture(i):
            current_texture_num = i
            update_display()
            print(f"Loaded texture t{current_texture_num}.png")
            return
    
    # If no previous textures, wrap to end
    for i in range(17, current_texture_num, -1):
        if load_texture(i):
            current_texture_num = i
            update_display()
            print(f"Wrapped to texture t{current_texture_num}.png")
            return
    
    print("No previous textures found")

# --- Event handlers ---
def on_move(event):
    """Handle mouse movement - show cursor"""
    if event.inaxes != ax or event.xdata is None or event.ydata is None:
        cursor_box.set_visible(False)
        fig.canvas.draw_idle()
        return
    
    cx, cy = int(event.xdata), int(event.ydata)
    cursor_box.set_visible(True)
    cursor_box.set_xy((cx - (stick_w * cursor_scale) / 2, cy - (stick_h * cursor_scale) / 2))
    cursor_box.set_width(stick_w * cursor_scale)
    cursor_box.set_height(stick_h * cursor_scale)
    fig.canvas.draw_idle()

def on_click(event):
    """Handle mouse click - select texture area"""
    if event.inaxes != ax or event.xdata is None or event.ydata is None:
        return
    
    cx, cy = int(event.xdata), int(event.ydata)
    
    # Store click data
    click_data = {
        'x': cx,
        'y': cy,
        'rotation': current_rotation,
        'scale': cursor_scale,
        'crop_w': crop_w,
        'crop_h': crop_h,
        'filename': f"t{current_texture_num}.png"  # Store filename for reference
    }
    
    # Initialize texture data if needed (use string key for JSON compatibility)
    texture_key = str(current_texture_num)
    if texture_key not in texture_data:
        texture_data[texture_key] = []
    
    texture_data[texture_key].append(click_data)
    
    # Visual feedback
    ax.plot(cx, cy, 'go', markersize=8)
    print(f"Selected area: Texture t{current_texture_num}.png, ({cx}, {cy}), Rot: {current_rotation}°, Scale: {cursor_scale:.1f}x")
    fig.canvas.draw_idle()

def on_key(event):
    """Handle keyboard input"""
    global current_rotation, cursor_scale
    
    if event.key == 'right':
        # Rotate 90 degrees clockwise
        current_rotation = (current_rotation + 90) % 360
        update_display()
        print(f"Rotated to {current_rotation}°")
        
    elif event.key == 'up':
        # Increase cursor scale
        cursor_scale = min(3.0, cursor_scale + 0.1)
        print(f"Cursor scale: {cursor_scale:.1f}x")
        
    elif event.key == 'down':
        # Decrease cursor scale
        cursor_scale = max(0.3, cursor_scale - 0.1)
        print(f"Cursor scale: {cursor_scale:.1f}x")
        
    elif event.key == ' ':  # Spacebar
        # Next texture
        next_texture()
        
    elif event.key == 'left':
        # Previous texture
        previous_texture()
        
    elif event.key == 'enter':
        # Save and exit
        save_texture_config()
        print("\nTEXTURE PREPROCESSING SUMMARY:")
        for tex_num, selections in texture_data.items():
            print(f"  Texture t{tex_num}.png: {len(selections)} selections")
        print(f"\nConfig saved to: {output_config_file}")
        print("Ready for stick generation!")
        plt.close()
        
    elif event.key == 'backspace':
        # Remove last selection for current texture
        texture_key = str(current_texture_num)
        if texture_key in texture_data and texture_data[texture_key]:
            removed = texture_data[texture_key].pop()
            print(f"Removed selection: ({removed['x']}, {removed['y']})")
            update_display()

# --- Main execution ---
def main():
    print(" TEXTURE PREPROCESSING TOOL - AI5 DATASET")
    print("=" * 60)
    print(" Source: datasets/tex2/ (t1.png to t17.png)")
    print(" Output: datasets/AI5/texture_selections_config.json")
    print("=" * 60)
    print("Controls:")
    print("  Click: Select texture area")
    print("  ➡️  Right Arrow: Rotate 90° clockwise")
    print("  ⬆️  Up Arrow: Increase cursor scale")
    print("  ⬇️  Down Arrow: Decrease cursor scale")
    print("  ⏯️  Spacebar: Next texture")
    print("  ⬅️  Left Arrow: Previous texture")
    print("  ⌫  Backspace: Remove last selection")
    print("  ⏎  Enter: Save and exit")
    print("=" * 60)
    
    # Check if texture directory exists
    if not os.path.exists(texture_base_path):
        print(f" Texture directory not found: {texture_base_path}")
        print(" Please ensure the datasets/tex2 folder exists with t1.png to t17.png files")
        return
    
    # Load existing config
    load_texture_config()
    
    # Load first texture
    if not load_texture(current_texture_num):
        print("Could not load any textures. Check your texture path!")
        print(f"Looking for files like: {texture_base_path}/t1.png")
        return
    
    # Set up event handlers
    fig.canvas.mpl_connect('motion_notify_event', on_move)
    fig.canvas.mpl_connect('button_press_event', on_click)
    fig.canvas.mpl_connect('key_press_event', on_key)
    
    # Initial display
    update_display()
    
    print(f" Starting with texture t{current_texture_num}.png")
    print(" Click on areas you want to use for stick generation!")
    
    # Show the interface
    plt.show()

if __name__ == "__main__":
    main()


🎯 Done! 0 crops saved to: C:\Users\theod\Documents\UNI\thesis\DATA_tiles1\stick_crops_m2
